Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Set constants for loops
NUM_TRIES = 12
TIME_SLEEP = 5 # in seconds

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

with open('config/config.json', 'r') as f:
    dic_config = json.load(f)
    OPEN_API_KEY = dic_config['OPEN_API_KEY']

# Initialize an empty list to store the stocks
stocks_list = []

# Open the text file in read mode
with open('stocks.txt', 'r') as file:
    # Read each line of the file
    for line in file:
        # Strip any leading or trailing whitespace and append the line to the list
        stocks_list.append(line.strip())

df_portfolio = pd.read_csv('config/portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

client = OpenAI(api_key=OPEN_API_KEY)

### Stocks Data

In [3]:
# stocks_list = ['MSFT', 'SHOP', 'AMD', 'NVDA', 'AAPL', 'OKTA', 'SHOP', 'AMAT', 'TTWO', 'SNOW', 'SOXX', 'BA', 'DOCN', 'MCD', 'LULU', 'CSCO', 'ORCL', 'AMZN', 'ASML']
stocks_list = ['MSFT']

yf_handler = YFHandler(stock_list=stocks_list, schemas=schemas)

df_stocks = yf_handler.import_stocks()
df_cashflow = yf_handler.import_cashflow()
df_income_stmt = yf_handler.import_income_stmt()

# Write to CSV
# Technically this will be done by the FileHandler but just to be safe
df_stocks.to_csv('openai_upload_files/df_stocks.csv', index=False)
df_cashflow.to_csv('openai_upload_files/df_cashflow.csv', index=False)
df_income_stmt.to_csv('openai_upload_files/df_income_stmt.csv', index=False)

file_stocks = FileHandler(
    df=df_stocks,
    dic_file=dic_files,
    file_name='df_stocks.csv',
    dic_file_name=OPENAI_DIC_FILE_NAME,
    file_path=FILE_PATH,
    dic_file_path=FILE_PATH,
    client=client
)

file_cashflow = FileHandler(
    df=df_cashflow,
    dic_file=dic_files,
    file_name='df_cashflow.csv',
    dic_file_name=OPENAI_DIC_FILE_NAME,
    file_path=FILE_PATH,
    dic_file_path=FILE_PATH,
    client=client
)

file_income_stmt = FileHandler(
    df=df_income_stmt,
    dic_file=dic_files,
    file_name='df_income_stmt.csv',
    dic_file_name=OPENAI_DIC_FILE_NAME,
    file_path=FILE_PATH,
    dic_file_path=FILE_PATH,
    client=client
)

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longe

In [4]:
# Update files
file_stocks.update_openai_file(dic_file=dic_files)
file_cashflow.update_openai_file(dic_file=dic_files)
file_income_stmt.update_openai_file(dic_file=dic_files)

file name: df_stocks.csv, file id: file-OXKIHrZZO9cFmKBK10bQJEiN has been deleted
file name: df_stocks.csv is uploaded, new file id: file-8HMrW8AwgEioBZDIBzLmRaFe
openai_upload_files/openai_files.json file has been updated
file name: df_cashflow.csv, file id: file-fHOLspcvscOYTfw2gq592se5 has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-Ut8koRazlXBAy5kd8d1kZajT
openai_upload_files/openai_files.json file has been updated
file name: df_income_stmt.csv, file id: file-STDhlBj7fbH8Dc69AwA5cLdW has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-2lmcUPpxyncLog5xbm9DnDSb
openai_upload_files/openai_files.json file has been updated


In [5]:
with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
    'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
    json.dump(dic_assistants, json_file)
    print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
    client = client, 
    new=False,
    assistant_name = 'fin_analyst',
    dic_file = dic_assistants,
    dic_file_name = 'assistants.json',
    dic_file_path='config/'
    )

fin_consultant = AgentHandler(
    client = client,
    new=False,
    assistant_name = 'fin_consultant',
    dic_file = dic_assistants,
    dic_file_name = 'assistants.json',
    dic_file_path='config/'
    )

assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U


In [18]:
def analyze_stock(client: OpenAI, ticker: list):
	
	yf_handler = YFHandler(stock_list=ticker, schemas=schemas)

	df_stocks = yf_handler.import_stocks()
	df_cashflow = yf_handler.import_cashflow()
	df_income_stmt = yf_handler.import_income_stmt()

	# Write to CSV
	# Technically this will be done by the FileHandler but just to be safe
	df_stocks.to_csv('openai_upload_files/df_stocks.csv', index=False)
	df_cashflow.to_csv('openai_upload_files/df_cashflow.csv', index=False)
	df_income_stmt.to_csv('openai_upload_files/df_income_stmt.csv', index=False)

	file_stocks = FileHandler(
		df=df_stocks,
		dic_file=dic_files,
		file_name='df_stocks.csv',
		dic_file_name=OPENAI_DIC_FILE_NAME,
		file_path=FILE_PATH,
		dic_file_path=FILE_PATH,
		client=client
	)

	file_cashflow = FileHandler(
		df=df_cashflow,
		dic_file=dic_files,
		file_name='df_cashflow.csv',
		dic_file_name=OPENAI_DIC_FILE_NAME,
		file_path=FILE_PATH,
		dic_file_path=FILE_PATH,
		client=client
	)

	file_income_stmt = FileHandler(
		df=df_income_stmt,
		dic_file=dic_files,
		file_name='df_income_stmt.csv',
		dic_file_name=OPENAI_DIC_FILE_NAME,
		file_path=FILE_PATH,
		dic_file_path=FILE_PATH,
		client=client
	)

	# Update files on OpenAI
	file_stocks.update_openai_file(dic_file=dic_files)
	file_cashflow.update_openai_file(dic_file=dic_files)
	file_income_stmt.update_openai_file(dic_file=dic_files)

	# Have to manually update the tool_resources because the file_id can change
	dic_assistants['fin_analyst']['tool_resources'] = {
		'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
	}

	with open('config/assistants.json', 'w') as json_file:
		json.dump(dic_assistants, json_file)
		print(f'assistants.json file has been updated')

	fin_analyst = AgentHandler(
		client = client, 
		new=False,
		assistant_name = 'fin_analyst',
		dic_file = dic_assistants,
		dic_file_name = 'assistants.json',
		dic_file_path='config/'
		)

	fin_consultant = AgentHandler(
		client = client,
		new=False,
		assistant_name = 'fin_consultant',
		dic_file = dic_assistants,
		dic_file_name = 'assistants.json',
		dic_file_path='config/'
		)
	
	# prompt_start = f"This is your client. I want you to advice me on the stocks I list below.\
	# 	I want to know the stocks' performance, if they're overvalued or undervalued, and whether or not they'll be a good investment.\
	# 	I am looking for a long term (5+ years) investment and I have a relatively high risk tolerance.\
	# 	Stocks: {ticker}"

	prompt_start = f"This is the financial consultant.\
		The client wants advice on the following stocks.\
		I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
		You will be provided with the data of these stocks, use them as you see fit.\
		Stocks: {ticker}"

	thread = ThreadManager(
		client=client,
		prompt=prompt_start
	)

	thread.run_thread(
		assistant=fin_analyst,
		prompt=prompt_start,
		attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
	)

	next_prompt = thread.last_message

	thread.run_thread(
		assistant=fin_consultant,
		prompt=next_prompt
	)

	thread.delete_thread()

	return thread.last_message

In [25]:
recommendation = analyze_stock(client=client, ticker=['SNOW'])

print(recommendation)

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longe

file name: df_stocks.csv, file id: file-cj3OLGQjqJZMAlwcLo0gJ2CK has been deleted
file name: df_stocks.csv is uploaded, new file id: file-shEIH3Cy5PXBiE1WVS3ifFzJ
openai_upload_files/openai_files.json file has been updated
file name: df_cashflow.csv, file id: file-TdLq06R5NRADSn5I3bn2jGce has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-rw5w6QHbRCtID1bXzrZ6JDsS
openai_upload_files/openai_files.json file has been updated
file name: df_income_stmt.csv, file id: file-I0bYEPYzkJJmK2kNfuRbjK3a has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-AYj1AdSR2DvVO1idhQ7MD9Yk
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U

assistant > Here are the information gathered from the financial analyst: 

These are the stocks the client wants